In [ ]:
include("../Engine/Engine.jl")
using .Engine

using StatsBase, DataFrames
import PyPlot as plt
import Seaborn as sns

In [ ]:
sizes = ["10", "50", "100", "200", "500"]
sizes_ = [10, 50, 100, 200, 500]

measure = "Followers"

paths_JDD = ["/Users/cyrilvallez/Desktop/Thesis/Results/JDD_aggregates_study/$measure/aggregate_size_$size/data.jld2" for size in sizes]
paths_TE = ["/Users/cyrilvallez/Desktop/Thesis/Results/TE_aggregates_study/$measure/aggregate_size_$size/data.jld2" for size in sizes]

graphs_JDD = []
cascades_JDD = []
dfs_JDD = []

for path in paths_JDD
    g, c, df = load_data(path)
    push!(graphs_JDD, g)
    push!(cascades_JDD, c)
    push!(dfs_JDD, df)
end

graphs_TE = []
cascades_TE = []
dfs_TE = []

for path in paths_TE
    g, c, df = load_data(path)
    push!(graphs_TE, g)
    push!(cascades_TE, c)
    push!(dfs_TE, df)
end

In [ ]:
edges_JDD = []
for (g, df) in zip(graphs_JDD, dfs_JDD)
    push!(edges_JDD, edge_types(g, df, 0))
end

edges_TE = []
for (g, df) in zip(graphs_TE, dfs_TE)
    push!(edges_TE, edge_types(g, df, 0))
end

In [ ]:
using DataStructures

for partition in ["Before", "During", "After"]

    counts = OrderedDict("T to T" => [], "T to U" => [], "U to T" => [], "U to U" => [])
    counts_normalized = OrderedDict("T to T" => [], "T to U" => [], "U to T" => [], "U to U" => [])

    for edges in edges_JDD
        indices = edges["partition"] .== partition
        edges_types = edges["edge_type"][indices]
        counts_ = edges["count"][indices]
        counts_normalized_ = edges["count_normalized"][indices]
        for (count, count_normalized, type) in zip(counts_, counts_normalized_, edges_types)
            push!(counts[type], count)
            push!(counts_normalized[type], count_normalized)
        end
    end

    sizes_ = [10, 50, 100, 200, 500]

    plt.figure()
    for key in keys(counts)
        plt.plot(sizes_, counts_normalized[key], label=key, marker="+")
    end
    plt.xlabel("Size of aggregates")
    plt.ylabel("Normalized count of edges")
    plt.legend()
    plt.grid()
    plt.title(partition)

end






In [ ]:

regex = r"^AGG[0-9]+:"

Proportions = []
N_Aggs = []
N_Actors = []
partitions_ = ["Before COP26", "During COP26", "After COP26"]
labels = [split(t, " ")[1] for t in partitions_]


for partition in partitions_

    proportions = []
    N_aggs = []
    N_actors = []

    for (g, df) in zip(graphs_JDD, dfs_JDD)
        partitions, actions, actors = partitions_actions_actors(df)
        graph = g[findfirst(partition .== partitions)]
        actors = actors[findfirst(partition .== partitions)]

        simplifier = x -> Int(maximum(x) > 0)
        graph = simplifier.(graph)
        sum_agg = 0
        for (i, a1) in enumerate(actors), (j, a2) in enumerate(actors)
            if graph[i,j] == 1 && (occursin(regex, a1) || occursin(regex, a2))
                sum_agg += 1
            end
        end

        N_agg = 0
        for a1 in actors 
            if occursin(regex, a1)
                N_agg += 1
            end
        end

        push!(N_aggs, N_agg)
        push!(N_actors, length(actors))


        proportion = sum_agg / sum(graph) 
        push!(proportions, proportion)

    end


    push!(Proportions, proportions)
    push!(N_Aggs, N_aggs)
    push!(N_Actors, N_actors)

    # plt.figure()
    # plt.plot(sizes_ , proportions, marker="+")
    # plt.xlabel("Size of aggregates")
    # plt.ylabel("Proportion of aggregate edges")
    # # plt.legend()
    # plt.grid()
    # plt.title(partition)

    # plt.figure()
    # plt.plot(N_aggs ./ N_actors , proportions, marker="+")
    # plt.xlabel("Proportion of aggregate actors")
    # plt.ylabel("Proportion of aggregate edges")
    # # plt.legend()
    # plt.grid()
    # plt.title(partition)

end


plt.figure()
for i = 1:length(Proportions)
    plt.plot(sizes_, Proportions[i] .* 100, marker="+", label=labels[i])
end
plt.xlabel("Size of aggregates")
plt.ylabel("Edges involving aggregates (\\%)")
plt.legend()
plt.ylim(-2, 102)
plt.xlim(0, 510)
plt.grid()
plt.savefig("/Users/cyrilvallez/Desktop/Thesis/Figures/Aggregates_study/$measure/JDD_edges.pdf", bbox_inches="tight")
(min_, max_) = plt.ylim()



plt.figure()
for i = 1:length(Proportions)
    plt.plot(sizes_, N_Aggs[i] ./ N_Actors[i] .* 100, marker="+", label=labels[i])
end
plt.xlabel("Size of aggregates")
plt.ylabel("Actor aggregates (\\%)")
plt.legend()
plt.grid()
plt.ylim(-2, 102)
plt.xlim(0, 510)
plt.savefig("/Users/cyrilvallez/Desktop/Thesis/Figures/Aggregates_study/$measure/JDD_actors.pdf", bbox_inches="tight")


In [ ]:

regex = r"^AGG[0-9]+:"

Proportions = []
N_Aggs = []
N_Actors = []
partitions_ = ["Before COP26", "During COP26", "After COP26"]
labels = [split(t, " ")[1] for t in partitions_]


for partition in partitions_

    proportions = []
    N_aggs = []
    N_actors = []

    for (g, df) in zip(graphs_TE, dfs_TE)
        partitions, actions, actors = partitions_actions_actors(df)
        graph = g[findfirst(partition .== partitions)]
        actors = actors[findfirst(partition .== partitions)]

        simplifier = x -> Int(maximum(x) > 0)
        graph = simplifier.(graph)
        sum_agg = 0
        for (i, a1) in enumerate(actors), (j, a2) in enumerate(actors)
            if graph[i,j] == 1 && (occursin(regex, a1) || occursin(regex, a2))
                sum_agg += 1
            end
        end

        N_agg = 0
        for a1 in actors 
            if occursin(regex, a1)
                N_agg += 1
            end
        end

        push!(N_aggs, N_agg)
        push!(N_actors, length(actors))


        proportion = sum_agg / sum(graph) 
        push!(proportions, proportion)

    end

    sizes_ = [10, 50, 100, 200, 500]


    push!(Proportions, proportions)
    push!(N_Aggs, N_aggs)
    push!(N_Actors, N_actors)

    # plt.figure()
    # plt.plot(sizes_ , proportions, marker="+")
    # plt.xlabel("Size of aggregates")
    # plt.ylabel("Proportion of aggregate edges")
    # # plt.legend()
    # plt.grid()
    # plt.title(partition)

    # plt.figure()
    # plt.plot(N_aggs ./ N_actors , proportions, marker="+")
    # plt.xlabel("Proportion of aggregate actors")
    # plt.ylabel("Proportion of aggregate edges")
    # # plt.legend()
    # plt.grid()
    # plt.title(partition)

end


plt.figure()
for i = 1:length(Proportions)
    plt.plot(sizes_, Proportions[i] .* 100, marker="+", label=labels[i])
end
plt.xlabel("Size of aggregates")
plt.ylabel("Edges involving aggregates (\\%)")
plt.legend()
plt.ylim(-2, 102)
plt.xlim(0, 510)
plt.grid()
plt.savefig("/Users/cyrilvallez/Desktop/Thesis/Figures/Aggregates_study/$measure/TE_edges.pdf", bbox_inches="tight")



plt.figure()
for i = 1:length(Proportions)
    plt.plot(sizes_, N_Aggs[i] ./ N_Actors[i] .* 100, marker="+", label=labels[i])
end
plt.xlabel("Size of aggregates")
plt.ylabel("Actor aggregates (\\%)")
plt.legend()
plt.grid()
plt.ylim(-2, 102)
plt.xlim(0, 510)
plt.savefig("/Users/cyrilvallez/Desktop/Thesis/Figures/Aggregates_study/$measure/TE_actors.pdf", bbox_inches="tight")
